<a href="https://colab.research.google.com/github/LoPA607/DH302/blob/main/dh302.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, glob, re
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import glm
from statsmodels.genmod.families import NegativeBinomial
from statsmodels.tsa.stattools import grangercausalitytests
import zipfile
import os
import pandas as pd
import pandas as pd
import re
from datetime import datetime


In [ ]:
# 0. Install required libs (run once)
!pip install cdsapi xarray netCDF4 pandas numpy rioxarray --quiet

# If you prefer GEE approach instead of CDS, tell me and I'll provide that variant.


In [ ]:

from google.colab import files
import os, textwrap

print("If you already have a ~/.cdsapirc file, skip this cell. Otherwise paste your CDS API 'key:' value when prompted.")

key = input("Paste your CDS API key (format 'user:id'): ").strip()
cds_content = f"url: https://cds.climate.copernicus.eu/api\nkey: {key}\n" # Corrected API URL
open('/root/.cdsapirc', 'w').write(cds_content)
print("Wrote /root/.cdsapirc")

In [ ]:
import cdsapi
c = cdsapi.Client()

years = ['2011','2012','2013','2014','2015']

for y in years:
    print("Downloading year:", y)

    c.retrieve(
        'reanalysis-era5-land',
        {
            'format': 'netcdf',
            'variable': [
                '2m_temperature',
                'total_precipitation',
                '2m_dewpoint_temperature'
            ],
            'year': y,
            'month': [f"{m:02d}" for m in range(1,13)],
            'day': [f"{d:02d}" for d in range(1,32)],
            'time': '00:00',
            'area': [28.8, 77.0, 28.4, 77.4],  # Delhi box
        },
        f'delhi_era5_daily_{y}.nc'
    )

print("✔ All years downloaded successfully")


In [ ]:

years = ['2011','2012','2013','2014','2015']

for y in years:
    input_file = f"/content/delhi_era5_daily_{y}.nc"
    output_dir = f"era5_extracted_daily_{y}" # Unique directory for each year

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    print(f"Extracting {input_file} to {output_dir}...")
    with zipfile.ZipFile(input_file, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    print(f"Finished extracting for {y}")

print("\n✔ All daily ERA5 files extracted to year-specific directories.")


df = pd.read_csv("/content/new_delhi_dengue_data.csv")


df["outbreak_date"] = pd.to_datetime({
    "year": df["year"],
    "month": df["mon"],
    "day": df["day"]
})

df_outbreaks = df[[
    "sno", "state_ut", "district", "Disease", "Cases", "Deaths",
    "Latitude", "Longitude", "outbreak_date"
]].copy()

print("Outbreak dates loaded:", df_outbreaks.shape)


In [ ]:
df_outbreak=pd.DataFrame()

In [ ]:

df = pd.read_csv("/content/new_delhi_dengue_data.csv")

# Extract week number
df['week_num'] = df['week_of_outbreak'].apply(lambda x: int(re.search(r'\d+', x).group()))

# Compute epidemic week's Monday
df['outbreak_date'] = df.apply(
    lambda r: datetime.strptime(f"{int(r['year'])}-W{int(r['week_num'])}-1", "%Y-W%W-%w"),
    axis=1
)

print(df[['week_of_outbreak','outbreak_date']])


In [ ]:
import xarray as xr
import glob

paths = sorted(glob.glob("/content/era5_extracted_daily_*/data_0.nc"))
print("Found files:", paths)

dfs = []

for p in paths:
    ds = xr.open_dataset(p)
    df = ds.to_dataframe().reset_index()
    dfs.append(df)

climate = pd.concat(dfs, ignore_index=True)
print(climate.head())


In [ ]:
!pip install xarray netCDF4 statsmodels --quiet


In [ ]:
era_paths = sorted(glob.glob("/content/era5_extracted_daily_*/data_*.nc"))
print("ERA5 files found:", len(era_paths), "files")

dfs = []
for p in era_paths:
    ds = xr.open_dataset(p)
    df = ds.to_dataframe().reset_index()
    dfs.append(df)

climate = pd.concat(dfs, ignore_index=True)

# Auto-detect time column
time_col = "time" if "time" in climate.columns else (
           "valid_time" if "valid_time" in climate.columns else "date")

climate = climate.rename(columns={time_col: "time"})
climate["time"] = pd.to_datetime(climate["time"])

numeric_cols = climate.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = ["time"] + numeric_cols
climate = climate[numeric_cols]
# -------------------------------------------------------------

climate_daily = climate.groupby("time").mean().reset_index()

# unit conversions
climate_daily["temp_C"] = climate_daily["t2m"] - 273.15
climate_daily["dewpoint_C"] = climate_daily["d2m"] - 273.15
climate_daily["precip_mm"] = climate_daily["tp"] * 1000.0

# RH calc (Magnus)
A = 17.625; B = 243.04
def rh(T, Td):
    return (np.exp((A*Td)/(B+Td)) / np.exp((A*T)/(B+T))) * 100

climate_daily["RH"] = rh(climate_daily["temp_C"], climate_daily["dewpoint_C"])
climate_daily = climate_daily.sort_values("time").reset_index(drop=True)
print("Daily climate shape:", climate_daily.shape)
print("Daily time range:", climate_daily['time'].min(), "->", climate_daily['time'].max())


In [ ]:
outbreak = pd.read_csv("/content/new_delhi_dengue_data.csv", dtype=str)

outbreak["year"] = pd.to_numeric(outbreak["year"], errors="coerce").astype(int)

# Extract week number (33rd week -> 33)
outbreak["week_num"] = outbreak["week_of_outbreak"].apply(lambda w: int(re.search(r"\d+", str(w)).group()))

def iso_week_to_monday(y, w):
    return datetime.fromisocalendar(int(y), int(w), 1)

outbreak["outbreak_date"] = outbreak.apply(lambda r: iso_week_to_monday(r["year"], r["week_num"]), axis=1)

outbreak["Cases"] = pd.to_numeric(outbreak.get("Cases", 0), errors="coerce").fillna(0).astype(float)

print("Outbreak rows:", outbreak.shape[0])
print("Outbreak date range:", outbreak['outbreak_date'].min(), "->", outbreak['outbreak_date'].max())


In [ ]:
windows = []

for idx, r in outbreak.iterrows():
    start = r["outbreak_date"] - timedelta(days=56)
    end = r["outbreak_date"] - timedelta(days=1)

    win = climate_daily[
        (climate_daily["time"] >= start) &
        (climate_daily["time"] <= end)
    ].copy()

    win["outbreak_idx"] = idx
    win["outbreak_date"] = r["outbreak_date"]

    for col in outbreak.columns:
        win[col] = r[col]

    windows.append(win)

df = pd.concat(windows, ignore_index=True)

print("Window dataset shape:", df.shape)

In [ ]:

df = df.sort_values(["outbreak_idx", "time"]).reset_index(drop=True)

df["precip_roll3_std"] = (
    df.groupby("outbreak_idx")["precip_mm"]
      .rolling(3, min_periods=1)
      .std().reset_index(0, drop=True)
      .fillna(0)
)

df["precip_7sum"] = (
    df.groupby("outbreak_idx")["precip_mm"]
      .rolling(7, min_periods=1)
      .sum().reset_index(0, drop=True)
)

median7 = df["precip_7sum"].median()
df["stagnation"] = (df["precip_7sum"] < median7).astype(int)

# Mosquito suitability
def temp_suit(T): return np.clip(1 - ((T - 28)/8)**2, 0, 1)
def rain_suit(p): return np.clip(p / 30, 0, 1)
def rh_suit(r):   return np.clip((r - 50)/50, 0, 1)

df["temp_suit"] = df["temp_C"].apply(temp_suit)
df["rain_suit"] = df["precip_mm"].apply(rain_suit)
df["rh_suit"] = df["RH"].apply(rh_suit)

df["MSI"] = (
    0.4*df["temp_suit"] +
    0.3*df["rain_suit"] +
    0.3*df["rh_suit"]
).clip(0,1)

# Save daily window features
df.to_csv("final_daily_features.csv", index=False)
print("Saved final_daily_features.csv")
print(df['MSI'])

In [ ]:
os.makedirs("outbreak_plots", exist_ok=True)

for oid in df["outbreak_idx"].unique():
    sub = df[df["outbreak_idx"] == oid].sort_values("time")
    date = sub["outbreak_date"].iloc[0].date()
    base = f"outbreak_{oid}_{date}"

    for var in ["temp_C", "RH", "precip_mm", "MSI"]:
        plt.figure(figsize=(10,4))
        plt.plot(sub["time"], sub[var])
        plt.title(f"{base} — {var}")
        plt.xlabel("Date"); plt.ylabel(var)
        plt.grid(True)
        plt.savefig(f"outbreak_plots/{base}_{var}.png")
        plt.close()

print("All outbreak plots saved → /outbreak_plots/")


In [ ]:
# ----------------------------------------
# 6) Weekly aggregation for causality (FIXED)
# ----------------------------------------

# Weekly MSI only from available days
dfw = df.copy()
dfw["week_start"] = dfw["time"].dt.to_period("W").apply(lambda r: r.start_time)
weekly_msi = dfw.groupby("week_start")["MSI"].mean().reset_index()
weekly_msi = weekly_msi.rename(columns={"week_start":"date"})

# Weekly dengue cases
outbreak["Cases"] = pd.to_numeric(outbreak["Cases"], errors="coerce").fillna(0)
weekly_cases = outbreak[["outbreak_date","Cases"]].rename(columns={"outbreak_date":"date"})

# Merge only overlapping weeks
series = pd.merge(weekly_msi, weekly_cases, on="date", how="inner")
series = series.sort_values("date").reset_index(drop=True)

print("\nFinal weekly series:")
print(series.head(), "\n")
print(series.tail())


In [ ]:
import seaborn as sns

os.makedirs("outbreak_summary_plots", exist_ok=True)

for oid in df["outbreak_idx"].unique():
    sub = df[df["outbreak_idx"] == oid].sort_values("time")
    date = sub["outbreak_date"].iloc[0].date()
    title = f"Outbreak {oid} — {date}"

    fig, axes = plt.subplots(4, 1, figsize=(12, 12), sharex=True)

    sns.lineplot(ax=axes[0], x=sub["time"], y=sub["temp_C"])
    axes[0].set_ylabel("Temp °C")

    sns.lineplot(ax=axes[1], x=sub["time"], y=sub["RH"])
    axes[1].set_ylabel("RH %")

    sns.lineplot(ax=axes[2], x=sub["time"], y=sub["precip_mm"])
    axes[2].set_ylabel("Rain (mm)")

    sns.lineplot(ax=axes[3], x=sub["time"], y=sub["MSI"])
    axes[3].set_ylabel("MSI")
    axes[3].set_xlabel("Date")

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()
    plt.savefig(f"outbreak_summary_plots/outbreak_{oid}_{date}.png")
    plt.close()


In [ ]:
df_heat = df.copy()
df_heat["date"] = df_heat["time"].dt.date
df_heat = df_heat.pivot_table(values="MSI",
                              index=df_heat["time"].dt.year,
                              columns=df_heat["time"].dt.strftime("%m-%d"),
                              aggfunc="mean")

plt.figure(figsize=(20,4))
sns.heatmap(df_heat, cmap="RdYlGn", cbar_kws={"label": "MSI"})
plt.title("Mosquito Suitability Heatmap")
plt.xlabel("Day of Year")
plt.ylabel("Year")
plt.tight_layout()
plt.savefig("MSI_heatmap.png")
plt.show()


In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df[["temp_C","RH","precip_mm","MSI","precip_7sum","precip_roll3_std"]].corr(),
            annot=True, cmap="coolwarm")
plt.title("Feature Correlation Matrix")
plt.tight_layout()
plt.savefig("correlation_matrix.png")
plt.show()


In [ ]:
plt.figure(figsize=(14,8))
lags = 6
for i in range(1, lags+1):
    plt.subplot(2,3,i)
    plt.scatter(series["MSI"], series["MSI"].shift(i), s=20)
    plt.title(f"MSI vs MSI(lag={i})")
    plt.xlabel("MSI")
    plt.ylabel(f"MSI lag {i}")

plt.tight_layout()
plt.savefig("lag_plots_msi.png")
plt.show()


In [ ]:
print(df.columns.tolist())


In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# list of outbreaks
outbreaks = df["outbreak_idx"].unique()
outbreaks = np.sort(outbreaks)

print("Outbreaks detected:", outbreaks)

for oid in outbreaks:
    sub = df[df["outbreak_idx"] == oid].copy()

    # select numeric feature columns
    features = ["temp_C", "RH", "precip_mm", "precip_7sum",
                "precip_roll3_std", "MSI"]

    corr = sub[features].corr()

    # plot heatmap
    plt.figure(figsize=(8,6))
    sns.heatmap(corr, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
    plt.title(f"Correlation Matrix – Outbreak {oid}")
    plt.tight_layout()
    plt.show()
    # save
    filename = f"correlation_outbreak_{oid}.png"
    plt.savefig(filename)
    plt.close()

    print(f"Saved: {filename}")


In [ ]:
display(df)

In [ ]:


# Load MODIS LAI
lai = pd.read_csv("/content/Delhi_LAI_8day.csv")

lai['date'] = pd.to_datetime(lai['date'])
lai = lai[['date','LAI']].sort_values('date')

# DAILY interpolation
lai_daily = lai.set_index('date').resample('D').interpolate(method='linear')

lai_daily = lai_daily.reset_index()
print(lai_daily.head(), lai_daily.tail())

lai_daily.to_csv("Delhi_LAI_daily.csv", index=False)
print("Saved Delhi_LAI_daily.csv")

In [ ]:

# LOAD FINAL MERGED DATA
# -----------------------------
df = pd.read_csv("/content/final_with_LAI.csv", parse_dates=["time"])
df['outbreak_date'] = pd.to_datetime(df['outbreak_date'])

# -----------------------------
# FIX LAI (missing values forward/backward fill)
# -----------------------------
df["LAI"] = df["LAI"].replace({None: np.nan})
df["LAI"] = df["LAI"].astype(float)
df["LAI"] = df["LAI"].interpolate().bfill().ffill()


# -----------------------------
# NORMALIZED SUITABILITY FUNCTIONS
# -----------------------------
def temp_suit(T):
    return np.clip(1 - ((T - 28) / 8) ** 2, 0, 1)

def rh_suit(r):
    return np.clip((r - 50) / 50, 0, 1)

def rain_suit(p):
    return np.clip(p / 30, 0, 1)

def lai_suit(l):
    return np.clip(l / 5, 0, 1)    # LAI 0–5 typical range


# -----------------------------
# COMPUTE SUITABILITIES
# -----------------------------
df["temp_suit"] = df["temp_C"].apply(temp_suit)
df["rh_suit"]   = df["RH"].apply(rh_suit)
df["rain_suit"] = df["precip_mm"].apply(rain_suit)
df["lai_suit"]  = df["LAI"].apply(lai_suit)

# -----------------------------
# NEW MSI (WITH LAI WEIGHTED)
# -----------------------------
df["MSI_new"] = (
    0.40 * df["temp_suit"] +
    0.20 * df["rh_suit"] +
    0.20 * df["rain_suit"] +
    0.20 * df["lai_suit"]
)

df["MSI_new"] = df["MSI_new"].clip(0, 1)


# ==============================
# PLOTTING FOR EACH OUTBREAK
# ==============================
os.makedirs("outbreak_plots_lai", exist_ok=True)

for oid in df["outbreak_idx"].unique():

    sub = df[df["outbreak_idx"] == oid].sort_values("time")
    outbreak_date = sub["outbreak_date"].iloc[0].date()

    fig, axes = plt.subplots(5, 1, figsize=(12, 14), sharex=True)

    sns.lineplot(ax=axes[0], x=sub["time"], y=sub["temp_C"])
    axes[0].set_ylabel("Temperature (°C)")

    sns.lineplot(ax=axes[1], x=sub["time"], y=sub["RH"])
    axes[1].set_ylabel("Relative Humidity (%)")

    sns.lineplot(ax=axes[2], x=sub["time"], y=sub["precip_mm"])
    axes[2].set_ylabel("Rainfall (mm)")

    sns.lineplot(ax=axes[3], x=sub["time"], y=sub["LAI"])
    axes[3].set_ylabel("LAI (vegetation index)")

    sns.lineplot(ax=axes[4], x=sub["time"], y=sub["MSI_new"])
    axes[4].set_ylabel("MSI (with LAI)")
    axes[4].set_xlabel("Date")

    plt.suptitle(f"Outbreak {oid} — {outbreak_date}")
    plt.tight_layout()

    plt.savefig(f"outbreak_plots_lai/outbreak_{oid}_{outbreak_date}_LAI.png")
    plt.show()
    plt.close()

print("All LAI-environment-MSI plots saved inside `outbreak_plots_lai/`")

In [ ]:
import itertools

INPUT_PATH = "/content/final_with_LAI.csv"
OUTDIR = "/content/outbreak_analysis_outputs"
os.makedirs(OUTDIR, exist_ok=True)
os.makedirs(os.path.join(OUTDIR,"plots_time_series"), exist_ok=True)
os.makedirs(os.path.join(OUTDIR,"plots_lags"), exist_ok=True)
os.makedirs(os.path.join(OUTDIR,"plots_corr"), exist_ok=True)

# Load dataframe
df = pd.read_csv(INPUT_PATH)
# normalize time column
time_col = None
for c in ["time","date","datetime","Date","date_time"]:
    if c in df.columns:
        time_col = c
        break
if time_col is None:
    raise ValueError("No time/date column found in dataframe columns: " + ", ".join(df.columns))
df[time_col] = pd.to_datetime(df[time_col])
df = df.sort_values(time_col).reset_index(drop=True)
df.rename(columns={time_col: "time"}, inplace=True)

# Check expected columns, try common names
for col in ["temp_C","RH","precip_mm","LAI","cases","outbreak_idx","outbreak_date"]:
    if col not in df.columns:
        # attempt alternatives
        if col=="temp_C":
            for alt in ["t2m","temp","temperature","T2M","temp_K"]:
                if alt in df.columns:
                    if alt=="temp_K" or alt=="t2m" or alt=="temp_K":
                        # convert K to C
                        df["temp_C"] = df[alt] - 273.15
                    else:
                        df["temp_C"] = df[alt]
                    break
        if col=="precip_mm":
            for alt in ["precip_mm","tp","precip","rain","rain_mm"]:
                if alt in df.columns:
                    if alt=="tp": df["precip_mm"] = df[alt]*1000.0
                    else: df["precip_mm"] = df[alt]
                    break
        if col=="RH":
            if "RH" not in df.columns and "rh" in df.columns:
                df["RH"] = df["rh"]
        if col=="LAI":
            for alt in ["LAI","lai","LAI_interp","LAI_daily"]:
                if alt in df.columns:
                    df["LAI"] = df[alt]
                    break
        if col=="cases":
            for alt in ["Cases","cases","cases_week","case_count"]:
                if alt in df.columns:
                    df["cases"] = df[alt]
                    break
        if col=="outbreak_idx":
            for alt in ["outbreak_idx","sno","id"]:
                if alt in df.columns:
                    df["outbreak_idx"] = df[alt]
                    break
        if col=="outbreak_date":
            for alt in ["outbreak_date","date_of_outbreak","outbreak_day"]:
                if alt in df.columns:
                    df["outbreak_date"] = pd.to_datetime(df[alt])
                    break

# Final check for necessary columns
missing = [c for c in ["temp_C","RH","precip_mm","LAI","outbreak_idx","outbreak_date"] if c not in df.columns]
if missing:
    raise ValueError("Missing required columns in input file: " + ", ".join(missing))

# Ensure numeric types
df["temp_C"] = pd.to_numeric(df["temp_C"], errors="coerce")
df["RH"] = pd.to_numeric(df["RH"], errors="coerce")
df["precip_mm"] = pd.to_numeric(df["precip_mm"], errors="coerce")
df["LAI"] = pd.to_numeric(df["LAI"], errors="coerce")

# fill/ interpolate small gaps in LAI
# Set 'time' column as index for time-weighted interpolation
df = df.set_index('time')
df["LAI"] = df["LAI"].interpolate(method="time").bfill().ffill()
df = df.reset_index() # Reset index for further operations

# Suitability functions
def temp_suit(T):
    return np.clip(1 - ((T - 28)/8)**2, 0, 1)
def rh_suit(r): return np.clip((r - 50)/50, 0, 1)
def rain_suit(p): return np.clip(p/30.0, 0, 1)
def lai_suit(l): return np.clip(l/5.0, 0, 1)

df["temp_suit"] = temp_suit(df["temp_C"])
df["rh_suit"] = rh_suit(df["RH"])
df["rain_suit"] = rain_suit(df["precip_mm"])
df["lai_suit"] = lai_suit(df["LAI"])

# New MSI with LAI (weights: temp 0.4, RH 0.2, rain 0.2, LAI 0.2)
df["MSI_new"] = (0.40*df["temp_suit"] + 0.20*df["rh_suit"] + 0.20*df["rain_suit"] + 0.20*df["lai_suit"]).clip(0,1)

# Save merged dataframe with MSI_new
OUT_CSV = os.path.join(OUTDIR, "final_with_LAI_MSI.csv")
df.to_csv(OUT_CSV, index=False)

# -----------------------
# A) Time-series plots per outbreak
# -----------------------
time_plots = []
for oid in sorted(df["outbreak_idx"].unique()):
    sub = df[df["outbreak_idx"]==oid].sort_values("time").copy()
    if sub.empty: continue
    out_date = pd.to_datetime(sub["outbreak_date"].iloc[0]).date()
    fig, axes = plt.subplots(5,1, figsize=(12,14), sharex=True)
    axes[0].plot(sub["time"], sub["temp_C"])
    axes[0].set_ylabel("Temp (°C)")
    axes[1].plot(sub["time"], sub["RH"])
    axes[1].set_ylabel("RH (%)")
    axes[2].plot(sub["time"], sub["precip_mm"])
    axes[2].set_ylabel("Precip (mm)")
    axes[3].plot(sub["time"], sub["LAI"])
    axes[3].set_ylabel("LAI")
    axes[4].plot(sub["time"], sub["MSI_new"])
    axes[4].set_ylabel("MSI_new")
    axes[4].set_xlabel("Date")
    fig.suptitle(f"Outbreak {oid} — {out_date}")
    fig.tight_layout(rect=[0,0,1,0.97])
    fn = os.path.join(OUTDIR,"plots_time_series", f"outbreak_{oid}_{out_date}_timeseries.png")
    fig.savefig(fn)
    plt.close(fig)
    time_plots.append(fn)

# -----------------------
# B) Lag scatter plots for each variable
# -----------------------
lags = [1,3,7,14]
vars_to_plot = {"temp_C":"Temperature","RH":"Relative Humidity","precip_mm":"Precipitation","LAI":"LAI","MSI_new":"MSI_new"}
lag_plots = []
for var in vars_to_plot:
    for L in lags:
        x = df[var].shift(L)
        y = df[var]
        mask = (~x.isna()) & (~y.isna())
        if mask.sum() < 10:
            continue
        plt.figure(figsize=(6,4))
        plt.scatter(x[mask], y[mask], s=10)
        plt.xlabel(f"{vars_to_plot[var]} (lag {L})")
        plt.ylabel(f"{vars_to_plot[var]} (today)")
        plt.title(f"{vars_to_plot[var]}: today vs lag {L}")
        fn = os.path.join(OUTDIR,"plots_lags", f"{var}_lag{L}.png")
        plt.tight_layout(); plt.savefig(fn); plt.close()
        lag_plots.append(fn)

# -----------------------
# C) Correlation matrices
# -----------------------
# 1) Instantaneous correlation
corr_vars = ["temp_C","RH","precip_mm","LAI","MSI_new"]
corr = df[corr_vars].corr()
# save heatmap
plt.figure(figsize=(6,5))
plt.imshow(corr, vmin=-1, vmax=1)
plt.colorbar(label="Pearson r")
plt.xticks(range(len(corr_vars)), corr_vars, rotation=45)
plt.yticks(range(len(corr_vars)), corr_vars)
# annotate
for i,j in itertools.product(range(len(corr_vars)), range(len(corr_vars))):
    plt.text(j, i, f"{corr.iloc[i,j]:.2f}", ha="center", va="center", color="white" if abs(corr.iloc[i,j])>0.5 else "black")
fn = os.path.join(OUTDIR,"plots_corr","correlation_matrix.png")
plt.title("Correlation matrix")
plt.tight_layout(); plt.savefig(fn); plt.close()

# 2) Lagged cross-correlation heatmap for MSI vs variables (lags up to 8 weeks (56 days) daily)
maxlag = 28  # days
lag_range = range(0, maxlag+1, 2)
heat = pd.DataFrame(index=lag_range, columns=corr_vars)
for L in lag_range:
    shifted = df["MSI_new"].shift(-L)  # MSI at t+L relative to predictor at t
    for v in corr_vars:
        heat.loc[L, v] = df[v].corr(shifted)
heat = heat.astype(float)

plt.figure(figsize=(8,6))
plt.imshow(heat.T, aspect='auto', vmin=-1, vmax=1)
plt.colorbar(label="Pearson r")
plt.yticks(range(len(corr_vars)), corr_vars)
plt.xticks(range(len(lag_range)), [str(l) for l in lag_range], rotation=90)
plt.xlabel("lag (days) for MSI = f(predictor at t)")
plt.title("Lagged correlation: predictor(t) vs MSI(t+lag)")
fn2 = os.path.join(OUTDIR,"plots_corr","lagged_correlation_heatmap.png")
plt.tight_layout(); plt.savefig(fn2); plt.close()

# Save heat dataframe
heat.to_csv(os.path.join(OUTDIR,"lagged_correlation_values.csv"))

# -----------------------
# D) Save outputs and small sample
# -----------------------
OUT_SUMMARY = {
    "final_csv": OUT_CSV,
    "final_with_msi_csv": os.path.join(OUTDIR,"final_with_LAI_MSI.csv"),
    "time_series_plots_count": len(time_plots),
    "lag_plots_count": len(lag_plots),
    "corr_plot": fn,
    "lagged_corr_plot": fn2,
    "plots_time_series": time_plots[:5]
}

# show a small preview of merged dataframe
print("Final sample with LAI and MSI:")
#print name of df
print(df.columns.tolist())
print(df.head(20))

print("All outputs written to:", OUTDIR)
print("Summary:", OUT_SUMMARY)


In [ ]:
from IPython.display import Image, display
import os

corr_matrix_path = os.path.join("/content/outbreak_analysis_outputs/plots_corr/", "correlation_matrix.png")

if os.path.exists(corr_matrix_path):
    display(Image(filename=corr_matrix_path))
else:
    print(f"Plot not found: {corr_matrix_path}")

In [ ]:
from IPython.display import Image, display
import os

lagged_corr_heatmap_path = os.path.join("/content/outbreak_analysis_outputs/plots_corr/", "lagged_correlation_heatmap.png")

if os.path.exists(lagged_corr_heatmap_path):
    display(Image(filename=lagged_corr_heatmap_path))
else:
    print(f"Plot not found: {lagged_corr_heatmap_path}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

df = pd.read_csv("/content/outbreak_analysis_outputs/final_with_LAI_MSI.csv", parse_dates=["time"])
df['outbreak_date'] = pd.to_datetime(df['outbreak_date'])

# Compute lag in days for each outbreak
df["lag_days"] = (df["time"] - df["outbreak_date"]).dt.days

# Keep only -56 to 0 days before outbreak
df = df[(df["lag_days"] >= -56) & (df["lag_days"] <= 0)]

# Ensure LAI filled
df["LAI"] = df["LAI"].astype(float).interpolate().bfill().ffill()

# Suitability calculations
def temp_suit(T): return np.clip(1 - ((T - 28) / 8)**2, 0, 1)
def rh_suit(r): return np.clip((r - 50) / 50, 0, 1)
def rain_suit(p): return np.clip(p / 30, 0, 1)
def lai_suit(l): return np.clip(l / 5, 0, 1)

df["temp_suit"] = temp_suit(df["temp_C"])
df["rh_suit"]   = rh_suit(df["RH"])
df["rain_suit"] = rain_suit(df["precip_mm"])
df["lai_suit"]  = lai_suit(df["LAI"])

df["MSI_new"] = (
    0.40*df["temp_suit"] +
    0.20*df["rh_suit"] +
    0.20*df["rain_suit"] +
    0.20*df["lai_suit"]
).clip(0,1)

# ================
# Plotting
# ================
os.makedirs("aligned_outbreak_plots", exist_ok=True)

params = {
    "temp_C": "Temperature (°C)",
    "RH": "Relative Humidity (%)",
    "precip_mm": "Rainfall (mm)",
    "LAI": "Leaf Area Index",
    "MSI_new": "MSI (with LAI)"
}

# One plot per parameter
for col, label in params.items():

    plt.figure(figsize=(14, 7))

    for oid in df["outbreak_idx"].unique():
        sub = df[df["outbreak_idx"] == oid]
        plt.plot(sub["lag_days"], sub[col], label=f"O{oid}", linewidth=2)

    plt.title(f"{label} — aligned by outbreak day")
    plt.xlabel("Days before outbreak")
    plt.ylabel(label)
    plt.axvline(0, linestyle="--", linewidth=2)  # outbreak day line
    plt.legend(title="Outbreak", fontsize=8)

    plt.tight_layout()
    plt.savefig(f"aligned_outbreak_plots/{col}_aligned.png")
    plt.show()
    plt.close()

print("All aligned plots saved in folder: aligned_outbreak_plots/")


In [ ]:
print("Date range:", df['time'].min(), "→", df['time'].max())
print("Unique outbreak dates:", df['outbreak_date'].unique())
print(df['outbreak_idx'].value_counts())


In [ ]:
# ====================================================================
# FULL LSTM PIPELINE WITH ALL PLOTS
# ====================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix,
    roc_curve, auc
)

import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ============================================================
# 1) Load Data
# ============================================================

PATH = "/content/outbreak_analysis_outputs/final_with_LAI_MSI.csv"
df = pd.read_csv(PATH)

df["time"] = pd.to_datetime(df["time"])
print("Original length:", len(df))

# ============================================================
# 2) Remove Duplicates
# ============================================================

df = df.drop_duplicates(subset=["time"]).sort_values("time")

# ============================================================
# 3) Reindex to daily continuous timeline
# ============================================================

full_range = pd.date_range(df["time"].min(), df["time"].max(), freq="D")
df = df.set_index("time").reindex(full_range)
df.index.name = "time"
df = df.ffill()

print("Final continuous length:", len(df))

# ============================================================
# 4) Outbreak Dates
# ============================================================

OUTBREAK_DATES = pd.to_datetime([
    "2013-09-16","2013-09-23","2013-10-07",
    "2015-08-10","2015-08-17","2015-08-24","2015-08-31",
    "2015-09-07","2015-09-14","2015-09-21","2015-09-28",
    "2015-10-05","2015-10-12","2015-10-19","2015-11-02"
])

print("Total outbreak dates:", len(OUTBREAK_DATES))

# ============================================================
# 5) Build 56-Day Windows
# ============================================================

WINDOW = 56
pos_windows = []

for d in OUTBREAK_DATES:
    start = d - pd.Timedelta(days=WINDOW)
    if start < df.index.min():
        print(f"⚠ Skipped {d} (window incomplete)")
        continue

    window = df.loc[start:d]
    if len(window) != WINDOW + 1:
        print(f"⚠ Skipped {d} (length {len(window)})")
        continue

    win = window.copy()
    win["label"] = 1
    win["outbreak_date"] = d
    pos_windows.append(win)

print("Positive windows:", len(pos_windows))

# ============================================================
# 6) Negative Windows
# ============================================================

neg_windows = []
needed = len(pos_windows)
non_outbreak_days = df.index.difference(OUTBREAK_DATES)

for d in non_outbreak_days:
    if len(neg_windows) >= needed:
        break

    start = d - pd.Timedelta(days=WINDOW)
    if start < df.index.min():
        continue

    window = df.loc[start:d]
    if len(window) != WINDOW + 1:
        continue

    win = window.copy()
    win["label"] = 0
    win["outbreak_date"] = d
    neg_windows.append(win)

print("Negative windows:", len(neg_windows))

# ============================================================
# 7) Combine Dataset
# ============================================================

all_windows = pd.concat(pos_windows + neg_windows)
all_windows = all_windows.reset_index().rename(columns={"index": "time"})
all_windows = all_windows.sort_values(["outbreak_date", "time"])

print("Dataset shape:", all_windows.shape)

# ============================================================
# 8) Build LSTM Sequences
# ============================================================

SEQ_LEN = WINDOW + 1
X_seq, y_seq = [], []

for date, g in all_windows.groupby("outbreak_date"):

    g_num = g.select_dtypes(include=[np.number]).copy()
    g_num = g_num.drop(columns=["label"], errors="ignore")

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(g_num)

    if len(X_scaled) == SEQ_LEN:
        X_seq.append(X_scaled)
        y_seq.append(g["label"].iloc[0])

X = np.array(X_seq)
y = np.array(y_seq)

print("X shape:", X.shape)
print("y shape:", y.shape)

# ============================================================
# 9) Train-Test Split
# ============================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, shuffle=True, random_state=42
)

# ============================================================
# 10) Build LSTM Model
# ============================================================

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQ_LEN, X.shape[2])),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.3),
    Dense(16, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=80,
    batch_size=16,
    callbacks=[es],
    verbose=1
)

# ============================================================
# 11) PLOTS
# ============================================================

# ---- LOSS PLOT ----
plt.figure(figsize=(7,5))
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.title("Training Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

# ---- ACCURACY PLOT ----
plt.figure(figsize=(7,5))
plt.plot(history.history["accuracy"], label="Train Acc")
plt.plot(history.history["val_accuracy"], label="Val Acc")
plt.title("Training Accuracy Curve")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()

# ============================================================
# 12) Predictions
# ============================================================

pred_probs = model.predict(X_test)
pred_labels = (pred_probs > 0.5).astype(int)

# ---- CONFUSION MATRIX ----
cm = confusion_matrix(y_test, pred_labels)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# ---- ROC CURVE ----
fpr, tpr, thresholds = roc_curve(y_test, pred_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(7,5))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.3f}")
plt.plot([0,1], [0,1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid()
plt.show()

# ============================================================
# 13) Classification Report
# ============================================================

print("\nCLASSIFICATION REPORT")
print(classification_report(y_test, pred_labels))

print("\nSample Predictions:")
for i in range(len(y_test)):
    print(f"True={y_test[i]}, Prob={pred_probs[i][0]:.3f}, Pred={pred_labels[i][0]}")

# ============================================================
# 14) Save Model
# ============================================================

model.save("/content/lstm_outbreak_model.h5")
print("\nModel saved as: /content/lstm_outbreak_model.h5")


In [ ]:
model.summary()

In [ ]:
# ============================================
# FULL CAUSALITY ANALYSIS PIPELINE
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import grangercausalitytests, ccf
import warnings
warnings.filterwarnings("ignore")

# ============================================
# 1) LOAD DATA
# ============================================
PATH = "/content/outbreak_analysis_outputs/final_with_LAI_MSI.csv"

df = pd.read_csv(PATH, parse_dates=["time"])
df = df.sort_values("time").reset_index(drop=True)

print("Loaded:", df.shape)

# outbreak_flag from outbreak_idx
df["outbreak_flag"] = (df["outbreak_idx"] > 0).astype(int)

# ============================================
# 2) SMOOTH OUTBREAK SIGNAL (required for Granger)
# ============================================
df["outbreak_smooth"] = (
    df["outbreak_flag"]
    .rolling(window=14, center=True, min_periods=1)
    .mean()
)

# Plot smoothed signal
plt.figure(figsize=(12,4))
plt.plot(df["time"], df["outbreak_smooth"])
plt.title("Smoothed Outbreak Signal")
plt.show()

# ============================================
# 3) CLEAN VARIABLES FOR ANALYSIS
# ============================================
causal_vars = ["temp_C", "RH", "precip_mm", "LAI", "MSI_new"]

for col in causal_vars:
    df[col] = df[col].interpolate().bfill().ffill()

# ============================================
# 4) GRANGER CAUSALITY FUNCTION
# ============================================
def run_granger(df, var, maxlag=30):

    print(f"\n\n===== GRANGER CAUSALITY: {var} → outbreak_smooth =====")

    data = df[["outbreak_smooth", var]].dropna()

    result = grangercausalitytests(data, maxlag=maxlag, verbose=False)

    for lag in [1, 7, 14, 21, 30]:
        if lag in result:
            p = result[lag][0]["ssr_chi2test"][1]
            print(f"Lag {lag:2d} → p = {p:.5f}")

# ============================================
# 5) RUN GRANGER FOR EACH VARIABLE
# ============================================
for var in causal_vars:
    run_granger(df, var)

# ============================================
# CORRELATION HEATMAP
# ============================================
heat_df = df[causal_vars ]

plt.figure(figsize=(10,6))
sns.heatmap(
    heat_df.corr(),
    annot=True,
    cmap="coolwarm",
    vmin=-1,
    vmax=1
)
plt.title("Correlation Heatmap (Environmental Factors vs Outbreak Risk)")
plt.show()
